In [ ]:
# The function implementation below is a modification version from Tensorflow
# Original code link: https://github.com/ashesh6810/DDWP-DA/blob/master/EnKF_DD_all_time.py


In [1]:
# Basic setting for Jupyter_notebook to import utils
import os
import sys

notebook_path = os.path.abspath("")
project_root = os.path.abspath(os.path.join(notebook_path, "../../"))

sys.path.append(project_root)

In [2]:
import netCDF4 as nc

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import scipy.io as sio
from tqdm import tqdm

In [4]:
import sys
import os
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import folder_utils

In [5]:
# Example usage

country = "GB"
data_folder = "data"
data_test_category = "test_data"
data_read_category = "raw_data"
data_save_category = "processed_data"
output_folder = "ERA5_DATA"
ddeg_out_lat = 0.25
ddeg_out_lon = 0.125

In [6]:
import tensorflow
import keras.backend as K
# from tensorflow.contrib import keras
# from tensorflow.contrib.keras import backend as K

#from data_manager import ClutteredMNIST
#from visualizer import plot_mnist_sample
#from visualizer import print_evaluation
#from visualizer import plot_mnist_grid
# import netCDF4 as nc

from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
import keras
from keras.callbacks import History
history = History()

import keras
from keras.layers import Conv2D, Conv2DTranspose, Cropping2D, Concatenate, ZeroPadding2D
from keras.models import load_model
from keras.layers import Input
from keras.models import Model
from keras.layers import Activation
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Dense

from utils import model_utils_tf
from bilinear_interpolation_1x import BilinearInterpolation

D:\anaconda3\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type

In [7]:
def get_initial_weights(output_size):
    b = np.zeros((2, 3), dtype='float32')
    b[0, 0] = 1
    b[1, 1] = 1
    W = np.zeros((output_size, 6), dtype='float32')
    weights = [W, b.flatten()]
    return weights

In [8]:
losses = []  # train loss
val_losses = []  # val_loss

In [9]:
def stn(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
    inputs = Input(shape=input_shape)
    conv1 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(pool2)
#     conv3 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv3)


    conv5 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv3)
#     conv5 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv5)
    
    locnet = Flatten()(conv5)
    locnet = Dense(500)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(200)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(100)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(50)(locnet)
    locnet = Activation('relu')(locnet)
    weights = get_initial_weights(50)
    locnet = Dense(6, weights=weights)(locnet)
    x = BilinearInterpolation(sampling_size)([inputs, locnet])


    up6 = keras.layers.Concatenate(axis=-1)([Convolution2D(32, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(x)), conv2])
    conv6 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv6)

    up7 = keras.layers.Concatenate(axis=-1)([Convolution2D(32, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(conv6)), conv1])
    conv7 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv7)



    conv10 = Convolution2D(1, 5, 5, activation='linear',border_mode='same')(conv7)

    model = Model(input=inputs, output=conv10)



    return model

In [10]:
model = stn()
model.compile(loss='mse', optimizer='adam')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  after removing the cwd from sys.path.
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  import sys
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 64, 32)   832         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 64, 32)   25632       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 16, 32, 32)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="relu", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (5, 5), activation="linear", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [11]:
fileList_train = []
fileList_validation =[]
fileList_test=[]
input_folder_path = folder_utils.find_folder(
    country, data_folder, data_save_category, output_folder
)

for year in range (1979,2021):
    file_path =  os.path.join(input_folder_path, f"era5_pressure_level_{year}_regrid_filter_850.nc")
    fileList_train.append (file_path)
    
fileList_validation.append(os.path.join(input_folder_path, f"era5_pressure_level_2021_regrid_filter_850.nc"))
fileList_test.append(os.path.join(input_folder_path, f"era5_pressure_level_2022_regrid_filter_850.nc"))


In [12]:
input_folder_path

'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA'

In [13]:
fileList_train

['F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_1979_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_1980_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_1981_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_1982_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_1983_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_1984_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_1985_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\p

In [14]:
M = 273.77817
sdev = 2.5819736

In [16]:
File=nc.Dataset(fileList_train[0])
File

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    regrid_method: bilinear
    Conventions: CF-1.6
    history: 2023-08-26 11:48:45 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data6/adaptor.mars.internal-1693050523.9571652-30807-11-9f2e4cee-f5dd-4cb7-bd14-d8cfa9322780.nc /cache/tmp/9f2e4cee-f5dd-4cb7-bd14-d8cfa9322780-adaptor.mars.internal-1693050488.2242575-30807-18-tmp.grib
    dimensions(sizes): time(8760), lat(32), lon(64)
    variables(dimensions): float32 t(time, lat, lon), int32 time(time), float64 lon(lon), float64 lat(lat)
    groups: 

In [17]:
Z=np.asarray(File['t'])
# Filter out zero and NaN values
Z.shape

(8760, 32, 64)

In [18]:
Z[0]

array([[261.73975, 261.64294, 261.54614, ..., 260.0619 , 260.09912,
        260.09912],
       [261.7046 , 261.5629 , 261.42117, ..., 259.8187 , 259.87036,
        259.87036],
       [261.7162 , 261.5668 , 261.41742, ..., 259.6002 , 259.62454,
        259.62454],
       ...,
       [261.46216, 261.42407, 261.38602, ..., 256.72403, 256.6516 ,
        256.6516 ],
       [261.79437, 261.66544, 261.5366 , ..., 256.6801 , 256.59903,
        256.59903],
       [262.21606, 262.0862 , 261.95657, ..., 256.4985 , 256.44095,
        256.44095]], dtype=float32)

In [16]:
Z[500][6]

array([270.38263, 270.43054, 270.47824, 270.53998, 270.6015 , 270.64752,
       270.69333, 270.71683, 270.7401 , 270.7686 , 270.79678, 270.85736,
       270.91766, 270.99496, 271.072  , 271.1141 , 271.156  , 271.10828,
       271.0604 , 270.9365 , 270.8125 , 270.69632, 270.58   , 270.54492,
       270.50964, 270.5439 , 270.57794, 270.60828, 270.63837, 270.62006,
       270.6015 , 270.54102, 270.4803 , 270.39453, 270.30853, 270.19342,
       270.07806, 269.91208, 269.74585, 269.51465, 269.2832 , 268.9912 ,
       268.69897, 268.4043 , 268.1093 , 267.90137, 267.6931 , 267.61523,
       267.53708, 267.53424, 267.53125, 267.5127 , 267.49405, 267.43845,
       267.38272, 267.3447 , 267.30658, 267.33893, 267.37112, 267.48727,
       267.6033 , 267.79468, 267.98605, 267.98605], dtype=float32)

In [19]:
trainN=np.size(Z,0)-300
trainN

8460

In [53]:
# mask = np.all(~np.isnan(Z), axis=(1, 2))
# Z = Z[mask]

In [105]:
Z

array([[[261.73975, 261.64294, 261.54614, ..., 260.0619 , 260.09912,
         260.09912],
        [261.7046 , 261.5629 , 261.42117, ..., 259.8187 , 259.87036,
         259.87036],
        [261.7162 , 261.5668 , 261.41742, ..., 259.6002 , 259.62454,
         259.62454],
        ...,
        [261.46216, 261.42407, 261.38602, ..., 256.72403, 256.6516 ,
         256.6516 ],
        [261.79437, 261.66544, 261.5366 , ..., 256.6801 , 256.59903,
         256.59903],
        [262.21606, 262.0862 , 261.95657, ..., 256.4985 , 256.44095,
         256.44095]],

       [[261.59326, 261.47308, 261.3529 , ..., 259.57553, 259.56784,
         259.56784],
        [261.50928, 261.3765 , 261.24365, ..., 259.3987 , 259.39575,
         259.39575],
        [261.43893, 261.31006, 261.18115, ..., 259.36264, 259.32166,
         259.32166],
        ...,
        [261.84695, 261.7033 , 261.5598 , ..., 256.76602, 256.68506,
         256.68506],
        [262.0327 , 261.89386, 261.75513, ..., 256.7269 , 256.62054,
   

In [20]:
Z=(Z-M)/sdev

In [21]:
Z

array([[[-4.6624885, -4.69998  , -4.737471 , ..., -5.312323 ,
         -5.2979035, -5.2979035],
        [-4.6761045, -4.7309823, -4.785872 , ..., -5.4065127,
         -5.3865023, -5.3865023],
        [-4.671613 , -4.7294693, -4.787326 , ..., -5.49114  ,
         -5.481708 , -5.481708 ],
        ...,
        [-4.7699986, -4.784749 , -4.799488 , ..., -6.6050787,
         -6.6331263, -6.6331263],
        [-4.6413317, -4.691269 , -4.7411704, ..., -6.622087 ,
         -6.653491 , -6.653491 ],
        [-4.47801  , -4.5283017, -4.5785108, ..., -6.6924243,
         -6.714716 , -6.714716 ]],

       [[-4.719222 , -4.765767 , -4.812312 , ..., -5.5006905,
         -5.503669 , -5.503669 ],
        [-4.751749 , -4.8031754, -4.8546257, ..., -5.5691724,
         -5.5703187, -5.5703187],
        [-4.778993 , -4.8289065, -4.878832 , ..., -5.5831428,
         -5.5990167, -5.5990167],
        ...,
        [-4.6209664, -4.676601 , -4.732176 , ..., -6.5888147,
         -6.620172 , -6.620172 ],
        [-4.

In [19]:
batch_size = 10
num_epochs = 8
lead=12
count=0

In [20]:
x_train=Z[0:trainN,:,:]
x_train=x_train.reshape([np.size(x_train,0),32,64,1])
y_train=Z[lead:trainN+lead,:,:]
y_train=y_train.reshape([np.size(y_train,0),32,64,1])

x_val= Z[trainN+lead:np.size(Z,0)-lead,:,:]
x_val=x_val.reshape([np.size(x_val,0),32,64,1])

y_val= Z[trainN+lead*2:np.size(Z,0),:,:]
y_val=y_val.reshape([np.size(y_val,0),32,64,1])

NameError: name 'trainN' is not defined

In [24]:
# Check for NaN values
if np.isnan(x_train).any() or np.isnan(y_train).any() or np.isnan(x_val).any() or np.isnan(y_val).any():
    print("Found NaN values in data, skipping this file:")

else:
    print("nan!!!")

nan!!!


In [42]:
Z[6609][0]

array([-5.923532 , -5.909916 , -5.896288 , -5.864884 , -5.8334675,
       -5.8014016, -5.7693353, -5.7389827, -5.708642 , -5.6810675,
       -5.6534925, -5.6470747, -5.6406684, -5.6531615, -5.6656547,
       -5.6648626, -5.664071 , -5.634912 , -5.6057534, -5.5641727,
       -5.52258  , -5.4768977, -5.4312034, -5.3766446, -5.322098 ,
       -5.266156 , -5.2102265, -5.165596 , -5.120966 , -5.0831437,
       -5.0453215, -5.0070972, -4.9688845, -4.939395 , -4.9099054,
       -4.8974123, -4.884919 , -4.8814797, -4.8780403, -4.879234 ,
       -4.880416 , -4.886432 , -4.892448 , -4.9000597, -4.90766  ,
       -4.913345 , -4.91903  , -4.928486 , -4.937941 , -4.9503756,
       -4.9628096, -4.9617577, -4.960694 , -4.940979 , -4.9212756,
       -4.8978143, -4.8743405, -4.8531837, -4.832015 , -4.805965 ,
       -4.779915 , -4.746596 , -4.713265 , -4.713265 ], dtype=float32)

In [25]:
nan_indices_x_train = (
    np.array([6610, 6610, 6610, 6780, 6780, 6780, 6780, 6780, 6827, 6827, 6827, 6827, 6827, 6848, 6848, 6848, 7950, 7950, 7950, 7950, 7950, 8166, 8166, 8166]),
    np.array([18, 19, 19, 7, 8, 8, 8, 8, 13, 14, 14, 14, 14, 4, 5, 5, 25, 26, 26, 26, 26, 1, 2, 2]),
    np.array([0, 0, 1, 0, 0, 1, 2, 3, 0, 0, 1, 2, 3, 0, 0, 1, 0, 0, 1, 2, 3, 0, 0, 1])
)

# Loop through the arrays in the tuple and print values
for row_idx, col_idx, channel_idx in zip(*nan_indices_x_train):
    print("Row Index:", row_idx)
    print("Column Index:", col_idx)
    print("Channel Index:", channel_idx)
    print(Z[row_idx][col_idx][channel_idx])
    print("-----")


Row Index: 6610
Column Index: 18
Channel Index: 0
-2.4731073
-----
Row Index: 6610
Column Index: 19
Channel Index: 0
-6.774948
-----
Row Index: 6610
Column Index: 19
Channel Index: 1
-6.774948
-----
Row Index: 6780
Column Index: 7
Channel Index: 0
-2.4583921
-----
Row Index: 6780
Column Index: 8
Channel Index: 0
-6.7855973
-----
Row Index: 6780
Column Index: 8
Channel Index: 1
-6.7855973
-----
Row Index: 6780
Column Index: 8
Channel Index: 2
-6.7855973
-----
Row Index: 6780
Column Index: 8
Channel Index: 3
-6.7855973
-----
Row Index: 6827
Column Index: 13
Channel Index: 0
-6.7793093
-----
Row Index: 6827
Column Index: 14
Channel Index: 0
-2.4690888
-----
Row Index: 6827
Column Index: 14
Channel Index: 1
-2.4690888
-----
Row Index: 6827
Column Index: 14
Channel Index: 2
-2.4690888
-----
Row Index: 6827
Column Index: 14
Channel Index: 3
-2.4690888
-----
Row Index: 6848
Column Index: 4
Channel Index: 0
1.811855
-----
Row Index: 6848
Column Index: 5
Channel Index: 0
-6.7841673
-----
Row In

In [26]:
Z[6848][4]

array([ 1.811855 ,  1.811855 ,  1.8171737,  1.8224925,  1.8282486,
        1.8300924,  1.83196  ,  1.8361086,  1.8402808, -2.4793954,
       -6.801341 , -6.790644 , -6.7845454, -2.4615362,  1.859192 ,
        1.8296788,  1.8002127,  1.7661372,  1.732109 ,  1.7097347,
        1.6874195,  1.686273 ,  1.6851739,  1.7021347,  1.7191548,
        1.7410799,  1.7630523,  1.7785594,  1.7941376,  1.8039596,
        1.8138406,  1.8220788,  1.8303642,  1.8390634,  1.8478216,
       -2.475235 , -6.800549 , -6.78464  , -6.7733054, -6.7471724,
       -6.7233205, -6.684399 , -6.64543  , -6.6012135, -6.5569615,
       -6.5198836, -6.4827704, -6.4544277, -6.4260373, -6.398486 ,
       -6.370899 , -6.3406296, -6.310336 , -6.2823715, -6.2543945,
       -6.225638 , -6.1968694, -6.1628175, -6.1287656, -6.092787 ,
       -6.0568204, -6.026208 , -5.9955955, -5.9955955], dtype=float32)

In [43]:
nan_indices_Z = np.where(np.isnan(Z))
print("NaN indices in x_train:")
print(nan_indices_Z)

NaN indices in x_train:
(array([6610, 6610, 6610, 6780, 6780, 6780, 6780, 6780, 6827, 6827, 6827,
       6827, 6827, 6848, 6848, 6848, 7950, 7950, 7950, 7950, 7950, 8166,
       8166, 8166], dtype=int64), array([18, 19, 19,  7,  8,  8,  8,  8, 13, 14, 14, 14, 14,  4,  5,  5, 25,
       26, 26, 26, 26,  1,  2,  2], dtype=int64), array([0, 0, 1, 0, 0, 1, 2, 3, 0, 0, 1, 2, 3, 0, 0, 1, 0, 0, 1, 2, 3, 0,
       0, 1], dtype=int64))


In [36]:
# Find and print indices of NaN values in x_train
nan_indices_x_train = np.where(np.isnan(x_train))
print("NaN indices in x_train:")
print(nan_indices_x_train)

# Find and print indices of NaN values in y_train
nan_indices_y_train = np.where(np.isnan(y_train))
print("NaN indices in y_train:")
print(nan_indices_y_train)

# Find and print indices of NaN values in x_val
nan_indices_x_val = np.where(np.isnan(x_val))
print("NaN indices in x_val:")
print(nan_indices_x_val)

# Find and print indices of NaN values in y_val
nan_indices_y_val = np.where(np.isnan(y_val))
print("NaN indices in y_val:")
print(nan_indices_y_val)

NaN indices in x_train:
(array([6610, 6610, 6610, 6780, 6780, 6780, 6780, 6780, 6827, 6827, 6827,
       6827, 6827, 6848, 6848, 6848, 7950, 7950, 7950, 7950, 7950, 8166,
       8166, 8166], dtype=int64), array([18, 19, 19,  7,  8,  8,  8,  8, 13, 14, 14, 14, 14,  4,  5,  5, 25,
       26, 26, 26, 26,  1,  2,  2], dtype=int64), array([0, 0, 1, 0, 0, 1, 2, 3, 0, 0, 1, 2, 3, 0, 0, 1, 0, 0, 1, 2, 3, 0,
       0, 1], dtype=int64), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64))
NaN indices in y_train:
(array([6598, 6598, 6598, 6768, 6768, 6768, 6768, 6768, 6815, 6815, 6815,
       6815, 6815, 6836, 6836, 6836, 7938, 7938, 7938, 7938, 7938, 8154,
       8154, 8154], dtype=int64), array([18, 19, 19,  7,  8,  8,  8,  8, 13, 14, 14, 14, 14,  4,  5,  5, 25,
       26, 26, 26, 26,  1,  2,  2], dtype=int64), array([0, 0, 1, 0, 0, 1, 2, 3, 0, 0, 1, 2, 3, 0, 0, 1, 0, 0, 1, 2, 3, 0,
       0, 1], dtype=int64), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [30]:
x_train.shape

(8460, 32, 64, 1)

In [31]:
y_train.shape

(8460, 32, 64, 1)

In [27]:
y_val.shape

(276, 32, 64, 1)

In [29]:
x_val.shape

(276, 32, 64, 1)

In [1]:
x_val

NameError: name 'x_val' is not defined

In [23]:
batch_size = 10
num_epochs = 8
lead=12
count=0
for loop in tqdm(fileList_train, desc="Processing files"):
    print('******************** counter*************',count)
    File=nc.Dataset(loop)
    Z=np.asarray(File['t'])
    
# # Create a mask to filter out samples with NaN values
#     mask = np.all(~np.isnan(Z), axis=(1, 2))
#     Z = Z[mask]
    
    trainN=np.size(Z,0)-300
    Z=(Z-M)/sdev
    
    
    x_train=Z[0:trainN,:,:]
    x_train=x_train.reshape([np.size(x_train,0),32,64,1])
    y_train=Z[lead:trainN+lead,:,:]
    y_train=y_train.reshape([np.size(y_train,0),32,64,1])
    
    x_val= Z[trainN+lead:np.size(Z,0)-lead,:,:]
    x_val=x_val.reshape([np.size(x_val,0),32,64,1])
    
    y_val= Z[trainN+lead*2:np.size(Z,0),:,:]
    y_val=y_val.reshape([np.size(y_val,0),32,64,1])

    if (count>0):
        
        model = stn()
        model.compile(loss='mse', optimizer='adam')
        # Modify the HDF5 file before loading weights
#         modify_h5_file('best_weights_lead12.h5')

        model.load_weights('best_weights_lead12.h5')
        hist = model.fit(x_train, y_train,
                       batch_size = batch_size,
             verbose=1,
             epochs = 20,
             validation_data=(x_val,y_val),shuffle=True,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights_lead'+str(lead)+'.h5', monitor='val_loss',
                                                    verbose=1, save_best_only=True,
                                                    save_weights_only=True, mode='auto', period=1),history]
             )
        losses.extend(hist.history['loss'])
        val_losses.extend(hist.history['val_loss'])

    else:
        hist = model.fit(x_train, y_train,
                       batch_size = batch_size,
             verbose=1,
             epochs = 20,
             validation_data=(x_val,y_val),shuffle=True,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights_lead'+str(lead)+'.h5', monitor='val_loss',
                                                    verbose=1, save_best_only=True,
                                                    save_weights_only=True, mode='auto', period=1),history]
             )
        losses.extend(hist.history['loss'])
        val_losses.extend(hist.history['val_loss'])


    count=count+1

Processing files:   0%|                                                                         | 0/42 [00:00<?, ?it/s]D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  after removing the cwd from sys.path.
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  import sys
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  
D:\anaconda3\envs\p

******************** counter************* 1


D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="relu", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="relu", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update you

Train on 8460 samples, validate on 276 samples
Epoch 1/20
8460/8460 [==============================] - 142s 17ms/step - loss: 4.0010 - val_loss: 0.7709

Epoch 00001: val_loss improved from inf to 0.77087, saving model to best_weights_lead12.h5
Epoch 2/20
8460/8460 [==============================] - 146s 17ms/step - loss: 3.9174 - val_loss: 0.7360

Epoch 00002: val_loss improved from 0.77087 to 0.73603, saving model to best_weights_lead12.h5
Epoch 3/20
8460/8460 [==============================] - 147s 17ms/step - loss: 3.7878 - val_loss: 0.8454

Epoch 00003: val_loss did not improve from 0.73603
Epoch 4/20
8460/8460 [==============================] - 149s 18ms/step - loss: 3.7374 - val_loss: 0.8584

Epoch 00004: val_loss did not improve from 0.73603
Epoch 5/20
8460/8460 [==============================] - 151s 18ms/step - loss: 3.6514 - val_loss: 0.8851

Epoch 00005: val_loss did not improve from 0.73603
Epoch 6/20
8460/8460 [==============================] - 156s 18ms/step - loss: 3.569

Processing files:   2%|█▍                                                          | 1/42 [17:27<11:55:37, 1047.25s/it]

...


Epoch 00011: val_loss did not improve from 0.86300
******************** counter************* 26
Train on 8484 samples, validate on 276 samples
Epoch 1/20
8484/8484 [==============================] - 588s 69ms/step - loss: 1.9170 - val_loss: 1.6491

Epoch 00001: val_loss improved from inf to 1.64907, saving model to best_weights_lead12.h5
Epoch 2/20
8484/8484 [==============================] - 580s 68ms/step - loss: 1.8590 - val_loss: 1.7832

Epoch 00002: val_loss did not improve from 1.64907
Epoch 3/20
8484/8484 [==============================] - 586s 69ms/step - loss: 1.8352 - val_loss: 1.7033

Epoch 00003: val_loss did not improve from 1.64907
Epoch 4/20
8484/8484 [==============================] - 585s 69ms/step - loss: 1.8030 - val_loss: 1.6535

Epoch 00004: val_loss did not improve from 1.64907
Epoch 5/20
8484/8484 [==============================] - 587s 69ms/step - loss: 1.7719 - val_loss: 1.7608

Epoch 00005: val_loss did not improve from 1.64907
Epoch 6/20
8484/8484 [=========

Processing files:  62%|██████████████████████████████████▋                     | 26/42 [23:28:46<18:11:11, 4091.95s/it]


Epoch 00006: val_loss did not improve from 1.64907
******************** counter************* 27
Train on 8460 samples, validate on 276 samples
Epoch 1/20
8460/8460 [==============================] - 600s 71ms/step - loss: 2.5096 - val_loss: 0.8686

Epoch 00001: val_loss improved from inf to 0.86855, saving model to best_weights_lead12.h5
Epoch 2/20
8460/8460 [==============================] - 594s 70ms/step - loss: 2.4404 - val_loss: 1.2330

Epoch 00002: val_loss did not improve from 0.86855
Epoch 3/20
4120/8460 [=============>................] - ETA: 4:57 - loss: 2.4373

Processing files:  62%|██████████████████████████████████▋                     | 26/42 [23:54:05<14:42:31, 3309.45s/it]


KeyboardInterrupt: 

In [24]:
fileList_train = []
fileList_validation =[]
fileList_test=[]
input_folder_path = folder_utils.find_folder(
    country, data_folder, data_save_category, output_folder
)

for year in range (2004,2021):
    file_path =  os.path.join(input_folder_path, f"era5_pressure_level_{year}_regrid_filter_850.nc")
    fileList_train.append (file_path)
    
fileList_validation.append(os.path.join(input_folder_path, f"era5_pressure_level_2021_regrid_filter_850.nc"))
fileList_test.append(os.path.join(input_folder_path, f"era5_pressure_level_2022_regrid_filter_850.nc"))


In [25]:
fileList_train

['F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_2004_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_2005_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_2006_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_2007_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_2008_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_2009_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\processed_data\\ERA5_DATA\\GB_ERA5_DATA\\era5_pressure_level_2010_regrid_filter_850.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\p

In [27]:
batch_size = 10
num_epochs = 8
lead=12
count=27
for loop in tqdm(fileList_train, desc="Processing files"):
    print('******************** counter*************',count)
    File=nc.Dataset(loop)
    Z=np.asarray(File['t'])
    
# # Create a mask to filter out samples with NaN values
#     mask = np.all(~np.isnan(Z), axis=(1, 2))
#     Z = Z[mask]
    
    trainN=np.size(Z,0)-300
    Z=(Z-M)/sdev
    
    
    x_train=Z[0:trainN,:,:]
    x_train=x_train.reshape([np.size(x_train,0),32,64,1])
    y_train=Z[lead:trainN+lead,:,:]
    y_train=y_train.reshape([np.size(y_train,0),32,64,1])
    
    x_val= Z[trainN+lead:np.size(Z,0)-lead,:,:]
    x_val=x_val.reshape([np.size(x_val,0),32,64,1])
    
    y_val= Z[trainN+lead*2:np.size(Z,0),:,:]
    y_val=y_val.reshape([np.size(y_val,0),32,64,1])

    if (count>0):
        
        model = stn()
        model.compile(loss='mse', optimizer='adam')
        # Modify the HDF5 file before loading weights
#         modify_h5_file('best_weights_lead12.h5')

        model.load_weights('best_weights_lead12.h5')
        hist = model.fit(x_train, y_train,
                       batch_size = batch_size,
             verbose=1,
             epochs = 20,
             validation_data=(x_val,y_val),shuffle=True,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights_lead'+str(lead)+'.h5', monitor='val_loss',
                                                    verbose=1, save_best_only=True,
                                                    save_weights_only=True, mode='auto', period=1),history]
             )
        losses.extend(hist.history['loss'])
        val_losses.extend(hist.history['val_loss'])

    else:
        hist = model.fit(x_train, y_train,
                       batch_size = batch_size,
             verbose=1,
             epochs = 20,
             validation_data=(x_val,y_val),shuffle=True,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights_lead'+str(lead)+'.h5', monitor='val_loss',
                                                    verbose=1, save_best_only=True,
                                                    save_weights_only=True, mode='auto', period=1),history]
             )
        losses.extend(hist.history['loss'])
        val_losses.extend(hist.history['val_loss'])


    count=count+1

Processing files:   0%|                                                                         | 0/17 [00:00<?, ?it/s]

******************** counter************* 27


D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  after removing the cwd from sys.path.
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  import sys
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 

Train on 8484 samples, validate on 276 samples
Epoch 1/20
8484/8484 [==============================] - 639s 75ms/step - loss: 1.8944 - val_loss: 1.7717

Epoch 00001: val_loss improved from inf to 1.77169, saving model to best_weights_lead12.h5
Epoch 2/20
8484/8484 [==============================] - 628s 74ms/step - loss: 1.8414 - val_loss: 1.6745

Epoch 00002: val_loss improved from 1.77169 to 1.67449, saving model to best_weights_lead12.h5
Epoch 3/20
8484/8484 [==============================] - 627s 74ms/step - loss: 1.7953 - val_loss: 1.6300

Epoch 00003: val_loss improved from 1.67449 to 1.62995, saving model to best_weights_lead12.h5
Epoch 4/20
8484/8484 [==============================] - 612s 72ms/step - loss: 1.7598 - val_loss: 1.6587

Epoch 00004: val_loss did not improve from 1.62995
Epoch 5/20
8484/8484 [==============================] - 619s 73ms/step - loss: 1.7252 - val_loss: 1.7714

Epoch 00005: val_loss did not improve from 1.62995
Epoch 6/20
8484/8484 [==================

Processing files:   6%|███▍                                                      | 1/17 [1:55:32<30:48:36, 6932.26s/it]

...


Epoch 00015: val_loss did not improve from 0.62570
******************** counter************* 42
Train on 8460 samples, validate on 276 samples
Epoch 1/20
8460/8460 [==============================] - 832s 98ms/step - loss: 1.7335 - val_loss: 2.3870

Epoch 00001: val_loss improved from inf to 2.38698, saving model to best_weights_lead12.h5
Epoch 2/20
8460/8460 [==============================] - 838s 99ms/step - loss: 1.6876 - val_loss: 2.4986

Epoch 00002: val_loss did not improve from 2.38698
Epoch 3/20
8460/8460 [==============================] - 837s 99ms/step - loss: 1.6591 - val_loss: 2.3719

Epoch 00003: val_loss improved from 2.38698 to 2.37185, saving model to best_weights_lead12.h5
Epoch 4/20
8460/8460 [==============================] - 837s 99ms/step - loss: 1.6474 - val_loss: 2.4586

Epoch 00004: val_loss did not improve from 2.37185
Epoch 5/20
8460/8460 [==============================] - 836s 99ms/step - loss: 1.6252 - val_loss: 2.4354

Epoch 00005: val_loss did not improve 

Processing files:  94%|█████████████████████████████████████████████████████▋   | 16/17 [27:39:57<2:01:57, 7317.90s/it]


Epoch 00008: val_loss did not improve from 2.37185
******************** counter************* 43
Train on 8484 samples, validate on 276 samples
Epoch 1/20
8484/8484 [==============================] - 853s 101ms/step - loss: 1.3450 - val_loss: 1.0542

Epoch 00001: val_loss improved from inf to 1.05418, saving model to best_weights_lead12.h5
Epoch 2/20
8484/8484 [==============================] - 857s 101ms/step - loss: 1.2957 - val_loss: 1.2057

Epoch 00002: val_loss did not improve from 1.05418
Epoch 3/20
8484/8484 [==============================] - 861s 101ms/step - loss: 1.2741 - val_loss: 1.0636

Epoch 00003: val_loss did not improve from 1.05418
Epoch 4/20
8484/8484 [==============================] - 861s 102ms/step - loss: 1.2827 - val_loss: 1.3855

Epoch 00004: val_loss did not improve from 1.05418
Epoch 5/20
8484/8484 [==============================] - 851s 100ms/step - loss: 1.2624 - val_loss: 1.0724

Epoch 00005: val_loss did not improve from 1.05418
Epoch 6/20
8484/8484 [====

Processing files: 100%|███████████████████████████████████████████████████████████| 17/17 [29:07:02<00:00, 6166.03s/it]


Epoch 00006: val_loss did not improve from 1.05418
